# Второе практическое задание
## Постановка задачи
Используя метод переменных направлений, решить краевую задачу: 
$$\left\{\begin{aligned}
\frac{\partial u}{\partial t}= \Delta u + \cos x \times e^{-t},\ ~ 0 < x < \pi,\ ~ 0 < y < \pi, t > 0\\
\frac{\partial u}{\partial x}\vert_{x=0} =  \frac{\partial u}{\partial x}\vert_{x=\pi} = 0\\
u\vert_{y = 0} = u\vert_{y = \pi} = 0\\
u\vert_{t = 0} = 0\\
\end{aligned}\\
\right. $$

# Точное решение
Решим вспомогательную задачу Штурма-Лиувилля в квадрате $\pi \times \pi$. 
$$\left\{\begin{aligned}
\Delta u + \lambda u = 0\\
u_x|_{x=0}=u_x|_{x=\pi}=0\\
u|_{y=0}=u|_{y=pi}=0
\end{aligned}\\
\right. $$
Отсюда получаем собственные функции области $v_{nm} = \cos nx \sin my$.

Решение исходной _однородной_ задачи нужно искать в виде $u(x, y, t) = \sum_{n,m} {T_{nm}(t)\cos nx ~\sin my}$
Неоднородность нужно разложить по собственным функциям:
$e^{-t}\cos x = \sum_n \sum_m f_{nm}(t) \cos nx ~\sin my$. Очевидно, что $f_{nm} = \delta_{1n}f_{1m}$. Поэтому остаётся только разложить экспоненту в ряд Фурье по синусам.
$||v_{nm}||^2 = \int^{\pi}_0 \cos ^ 2 nx~dx \int^{\pi}_0 \sin ^2 my~dy = \frac{\pi^2}{4}$ - это норма.

$$f^0_{nm} = \frac{1}{||v_{nm}||} \int^{\pi}_0 dy~\sin my~\int^{\pi}_0 dx~\cos nx = 2*\frac{1-(-1)^m}{\pi m}$$
Отсюда заключаем, что $m$ может быть только нечётным, т.е. $m = 2k-1 \Rightarrow f^0_{nm} = \frac{4}{\pi(2k-1)}$.

Как нас учит ММФ: $T_m(t) = \int^{\pi}_0 e^{-\lambda_m (t-\tau)}f_m(\tau) ~d\tau$, где $\lambda_m = m^2+n^2 = (2k-1)^2+1$

Задача Коши на временную часть:
$$\left\{\begin{aligned}
T'_{1m}(t) + \lambda_{1m} T_{1m}(t) = e^{-t}\frac{4}{\pi(2k-1)}\\
T_{1m}(0)=0
\end{aligned}\\
\right. $$
Эта задача имеет решение $T_{1m}(t) = \frac{4}{\pi(2k-1)}\frac{e^{-t}-e^{-\lambda t}}{\lambda-1}=\frac{4}{\pi}\frac{e^{-t}-e^{-\lambda t}}{(2k-1)^3}$
Точное решение: $$u(x, y, t) = \frac{4}{\pi} \sum_k \frac{e^{-t}-e^{-t\lambda}}{(2k-1)^3}\sin (2k-1)y~\cos x$$

# Разностная сетка
Вводим сетку на в области $\Omega = G \otimes [0, T]$:
$G = \{(x,y); 0 < x < \pi, ~ 0 < y < \pi\}$
За шаг по оси $X$ примем $h_x$ ($Nh_x = \pi$), аналогично за шаг по оси $y$ примем $h_y$ ($Kh_y = \pi$). Шаг по сетке времени обозначим за $\tau$. Тогда:
$$u_t\vert_{x = x_n, y = y_k} = \frac{u^{m+1}_{n, k} - u^{m}_{n, k}}{\tau}$$
$$u_{xx} + u_{yy} = \frac{u^{?}_{n+1,k} -2u^{?}_{n,k} + u^{?}_{n-1,k}}{h_x^2} + \frac{u^{?}_{n, k+1} -2u^{?}_{n,k} + u^{?}_{n,k-1}}{h^2_y}$$
При $t = t_m$ эта схема становится явной и условно устойчивой (аналог критерия Куранта: $\tau < \frac{c^2(h^2_x+h^2_y)}{2}$):
$$u_{xx} + u_{yy} = \frac{u^{m}_{n+1,k} -2u^{m}_{n,k} + u^{m}_{n-1,k}}{h_x^2} + \frac{u^{m}_{n, k+1} -2u^{m}_{n,k} + u^{m}_{n,k-1}}{h^2_y}$$

# Устойчивость
Поговорим про устойчивость данного решения. Если бы данная задача была одномерной, то для её решения было бы достаточно явной схемы по оси $x$, однако оная требует маленького шага по времени (т.е. является условно устойчивой). Неявные схемы, в которых оператор дифференцирования по пространственной переменной аппроксимируется на новом слое по времени безусловно устойчивы, но для них нужно решать систему линейных уравнений, что тоже увеличивает количество производимых алгоритмом действий. Для двумерной задачи применение явной схемы уже невероятно усложнено количеством действий. Более того, даже для современных вычислительных технологий это может быть неплохим вызовом, если учитывать такие тонкости, как время на одну операцию с плавающей запятой. Здесь я решаю задачу с помощью и без того медлительного и перегруженного высокоуровневого языка Python, где за секунду в лучшем случае выполняется порядка $10^6$ операций. Если вспомнить критерий Куранта, получится, что шаги по времени и по пространству связаны соотношением $\tau \leq \frac{c^2(h_x^2+h_y^2)}{2},$ откуда видим, что даже при адекватных шагах $h_x$ и $h_y$ шаг по времени $\tau$ должен быть колоссально маленьким, а их количество - безумно большим. Для того, чтобы избежать этого, вводится экономичная разностная схема, сочетающая в себе преимущества и явных, и неявных схем.

# Схема Писмена-Рекфорда
Каждый шаг по времени выполняется в 2 этапа:

1) $t_m \rightarrow t_{m+\frac{1}{2}}$

$u_{xx}\vert_{t_m} = \dots$ - явно

$u_{yy}\vert_{t_{m+\frac{1}{2}}} = \dots$ - неявно

$$\left\{\begin{aligned}
\frac{2}{\tau}\left( u^{m+\frac{1}{2}}_{n,k} - u^{m}_{n,k} \right) = \frac{1}{h_x^2}\left( u^{m}_{n+1,k} -2u^{m}_{n,k} + u^{m}_{n-1,k}\right) + \frac{1}{h^2_y}\left( u^{m+\frac{1}{2}}_{n, k+1} -2u^{m+\frac{1}{2}}_{n,k} + u^{m+\frac{1}{2}}_{n,k-1} \right) + f(x_n, y_k, t_{m+\frac{1}{2}})\\
\frac{u^{m}_{1, k} - u^{m}_{0, k}}{h_x} =  \frac{u^{m}_{N, k} - u^{m}_{N-1, k}}{h_x} = 0\\
u_{n, 0} = u_{n, K} = 0\\
c = 1, u^0_{n, k} = 0\\
\end{aligned}\\
\right. $$

Это приводится к следующему удобному виду:

$$\left\{\begin{aligned}
A^{m+\frac{1}{2}}u^{m+\frac{1}{2}}_{n, k-1} - C^{m+\frac{1}{2}}u^{m+\frac{1}{2}}_{n, k} + B^{m+\frac{1}{2}}u^{m+\frac{1}{2}}_{n, k+1} = -F^{m+\frac{1}{2}}\\
A^{m+\frac{1}{2}} = B^{m+\frac{1}{2}} = \frac{0.5\tau}{h^2_y}, ~C^{m+\frac{1}{2}} = \frac{\tau}{h^2_y}+1\\
F^{m+\frac{1}{2}} = \frac{0.5\tau}{h^2_x}\left( u^m_{n+1, k}-2u^m_{n,k}+u^m_{n-1,k}\right)+u^m_{n,k}+0.5\tau f(x_n, y_k, t_{m+\frac{1}{2}})
\end{aligned}
\right. $$

Данная система решается методом прогонки. После этого можно переходить от промежуточного слоя к следующему значению:

2) $t_{m+\frac{1}{2}} \rightarrow t_{m+1}$

$u_{xx}\vert_{t_{m+1}} = \frac{c^2}{h_x^2}\left( u^{m+1}_{n+1,k} -2u^{m+1}_{n,k} + u^{m+1}_{n-1,k}\right)$

$u_{yy}\vert_{t_{m+\frac{1}{2}}} = \frac{c^2}{h^2_y}\left( u^{m+\frac{1}{2}}_{n, k+1} -2u^{m+\frac{1}{2}}_{n,k} + u^{m+\frac{1}{2}}_{n,k-1} \right)$

$$\left\{\begin{aligned}
\frac{2}{\tau}\left( u^{m+1}_{n,k} - u^{m+\frac{1}{2}}_{n,k} \right) = \frac{1}{h_x^2}\left( u^{m+\frac{1}{2}}_{n+1,k} -2u^{m+\frac{1}{2}}_{n,k} + u^{m+\frac{1}{2}}_{n-1,k}\right) + \frac{1}{h^2_y}\left( u^{m+1}_{n, k+1} -2u^{m+1}_{n,k} + u^{m+1}_{n,k-1} \right) + f(x_n, y_k, t_{m})\\
\frac{u^{m}_{1, k} - u^{m}_{0, k}}{h_x} =  \frac{u^{m}_{N, k} - u^{m}_{N-1, k}}{h_x} = 0\\
u_{n, 0} = u_{n, K} = 0\\
c = 1, u^0_{n, k} = 0\\
\end{aligned}\\
\right. $$

Это приводится к следующему удобному виду:
$$\left\{\begin{aligned}
A^{m+1}u^{m+1}_{n, k-1} - C^{m+1}u^{m+1}_{n, k} + B^{m+1}u^{m+1}_{n, k+1} = -F^{m+1}\\
A^{m+1} = B^{m+1} = \frac{0.5\tau}{h^2_y}, ~C^{m+1} = \frac{\tau}{h^2_x}+1\\
F^{m+1} = \frac{0.5\tau}{h^2_x}\left( u^{m+\frac{1}{2}}_{n+1, k}-2u^{m+\frac{1}{2}}_{n,k}+u^{m+\frac{1}{2}}_{n-1,k}\right)+u^{m+\frac{1}{2}}_{n,k}+0.5\tau f(x_n, y_k, t_{m+1})
\end{aligned}\\
\right. $$


Аппроксимироваться должно с погрешностью $\Psi = O(h^2_x+h^2_y+\tau^2)$

In [121]:
%matplotlib notebook
import numpy as np
import plotly
import plotly.graph_objs as go
import warnings
from plotly.offline import iplot
import pylab

In [163]:


def f(x, y, t):
    return np.cos(x)*np.exp(-t)
    #return 0

N = 80
K = 80
M = 100
X_begin = 0; X_end = np.pi
Y_begin = 0; Y_end = np.pi
T_begin = 0; T_end = 1.7
hx = (X_end - X_begin)/(N-1)
hy = (Y_end - Y_begin)/(K-1)
ht = (T_end - T_begin)/(M-1)
x = np.linspace(X_begin, X_end, N)
y = np.linspace(Y_begin, Y_end, K)
T0 = 50
Ax = Bx = 0.5*ht/(hy)**2
Ay = By = 0.5*ht/(hx)**2
Cx = ht/hy**2 + 1
Cy = ht/hx**2 + 1

plotly.offline.init_notebook_mode(connected=True)
warnings.filterwarnings('ignore')
%matplotlib notebook
%config InlineBackend.figure_format = 'retina'

X, Y = np.meshgrid(x, y)
def plot_u(Y1, X1, u1, online=True, filename = '123.png'):
        '''
        Функция, который строит 3D график численного решения.
        '''
        X0 = X1
        Y0 = Y1
        u = u1
        data = [go.Surface(x = X0, y = Y0, z = u, colorscale = 'YlGnBu')]
        # Наведем красоту
        layout = go.Layout(
                    scene = dict(
                    xaxis = dict(
                        title='x',
                        gridcolor="rgb(255, 255, 255)",
                        zerolinecolor="rgb(255, 255, 255)",
                        showbackground=True,
                        range = [0, np.pi],
                        backgroundcolor="rgb(200, 200, 230)"),
                        
                    yaxis = dict(
                        title='y',
                        gridcolor="rgb(255, 255, 255)",
                        zerolinecolor="rgb(255, 255, 255)",
                        showbackground=True,
                        range = [0, np.pi],
                        backgroundcolor="rgb(230, 200,230)"),
                        
                    zaxis = dict(
                        title='u(x, y)',
                        gridcolor="rgb(255, 255, 255)",
                        zerolinecolor="rgb(255, 255, 255)",
                        range = [-np.max(u), np.max(u)],
                        showbackground=True,
                        backgroundcolor="rgb(230, 230,200)",),),
                    width=500, height=500, 
                    margin=dict(
                    r=20, b=10,
                    l=10, t=10)
                  )

        fig = go.Figure(data = data, layout = layout)
        if  online:
            return plotly.offline.iplot(fig, filename = filename)  # построение графика оффлайн влечет за собой
                                        # большой вес ноутбука, но построение почти мгновенное

In [164]:
la = np.zeros((N, K))
ua = np.zeros((T0+2, N, K))
ua[0] = la
for m in range(T0+1):
    t = m*ht#T_end-T_begin
    for n in range(N):
        for k in range(K):
            s = 0
            for i in range(1, 40):
                z = 2*i-1
                lm = z**2+1
                s+=4/np.pi*((np.exp(-t)-np.exp(-t*lm))/z**2)*np.sin(z*hy*k)*np.cos(hx*n)
            la[n][k] = s
    norm = 1 # max([max(i) for i in la])
    la = la.T/norm
    ua[m+1] = la
plot_u(Y, X, la)
print("t = ", t)

t =  0.8585858585858586


In [165]:
l = np.zeros((N, K))
u = np.zeros((T0+2, N, K))
u[0] = l
for m in np.arange(T0+1):
    lh = np.zeros((N, K))
    for n in np.arange(1, N - 1):
        F = np.zeros(K)
        alpha =np.zeros(K)
        beta = np.zeros(K)
        alpha[1] = 0 #-Bx/Cx
        beta[1] = 0 #-F[0]/Cx
        result = np.zeros(K)
        for k in np.arange(K):
            F[k] = ht/(2*hx*hx)*l[n-1][k]+(1-ht/hx**2)*l[n][k]+ht/(2*hx*hx)*l[n+1][k]+ht/2*f(hx*n, 0, (m+0.5)*ht)
        for k in np.arange(1, K-1):
            alpha[k+1] = -Bx/(Ax*alpha[k]-Cx)
            beta[k+1] = (-Ax*beta[k]-F[k])/(Ax*alpha[k]-Cx)
        result[K-1] = 0
        for k in range(K-2, 0, -1):
            result[k] = alpha[k+1]*result[k+1]+beta[k+1]
        lh[n] = result
    #for k in np.arange(K):
        #lh[0][k] = lh[1][k]
        #lh[N-1][k] = lh[N-2][k]
    
    l = l.T
    lh = lh.T
    for k in np.arange(1, K - 1):
        F = np.zeros(N)
        alpha =np.zeros(N)
        beta = np.zeros(N)
        alpha[1] = 1
        result = np.zeros(N)
        for n in np.arange(N):
            F[n] = ht/(2*hx*hx)*lh[k-1][n]+(1-ht/hx**2)*lh[k][n]+ht/(2*hx*hx)*lh[k+1][n]+ht/2*f(hx*n, 0, (m+1)*ht)
        for n in np.arange(1, N-1):
            alpha[n+1] = -Bx/(Ax*alpha[n]-Cx)
            beta[n+1] = (-Ax*beta[n]-F[n])/(Ax*alpha[n]-Cx)
        result[N-1] = beta[N-1]/(1-alpha[N-1])
        for n in range(N-2, 0, -1):
            result[n] = alpha[n+1]*result[n+1]+beta[n+1]
        result[0] = result[1]
        result[N-1] = result[N-2]
        l[k]= result
    for n in np.arange(N):
        l[0][n] = l[K-1][n] = 0
    
    l = l.T
    u[m+1] = l.T

plot_u(Y, X, l.T)
print("t = ", m*ht)

t =  0.8585858585858586


In [166]:
plot_u(Y, X, abs(l.T-la))

In [128]:
def show_evolution(u):
        '''
        Метод, который отображает анимацию эволюции u(x, y, t).
        '''
        
        layout = go.Layout(
                    scene = dict(
                    xaxis = dict(
                        title='x',
                        gridcolor="rgb(255, 255, 255)",
                        zerolinecolor="rgb(255, 255, 255)",
                        showbackground=True,
                        range = [0,np.pi],
                        backgroundcolor="rgb(200, 200, 230)"),
                        
                    yaxis = dict(
                        title='y',
                        gridcolor="rgb(255, 255, 255)",
                        zerolinecolor="rgb(255, 255, 255)",
                        showbackground=True,
                        range = [0,np.pi],
                        backgroundcolor="rgb(230, 200,230)"),
                        
                    zaxis = dict(
                        title='u(x, y)',
                        gridcolor="rgb(255, 255, 255)",
                        zerolinecolor="rgb(255, 255, 255)",
                        range = [-0.5, 0.5],
                        showbackground=True,
                        backgroundcolor="rgb(230, 230,200)",),),
                    width=500, height=500, 
                    margin=dict(
                    r=20, b=10,
                    l=10, t=10)
                  )
        # Create FigureWidget and add surface trace
        fig = go.Figure(layout = layout)
        surface = fig.add_surface(z=u[0], colorscale = 'YlGnBu')

        # Set axis ranges to fixed values to keep them from retting during animation
        fig.layout.scene.zaxis.range = [np.min(u), np.max(u)]
        fig.layout.scene.yaxis.range = [Y_begin, Y_end]
        fig.layout.scene.xaxis.range = [X_begin, X_end]

        frames = []
        for j in range(T0):
            frames.append(go.Frame(data=[{'type': 'surface','x': X, 'y': Y, 'z': u[j]}]))

        fig.frames = frames


        #display(iplot(fig))
        return fig


In [145]:
#fig = show_evolution(u)
figan = show_evolution(ua)

In [143]:
u.shape, ua.shape

((52, 10, 10), (52, 10, 10))